In [1]:
import pandas as pd
from collections import defaultdict
# temp1 = pd.read_csv("tram_scores.csv")
# temp2 = pd.read_csv("bus_scores.csv")=
import numpy as np
from collections import OrderedDict

In [2]:
dataframe = pd.read_csv("scores.csv")
direction = pd.read_csv("route_stop_sequence.csv")

In [3]:
direction.head()

,route_id,direction_id,stop_sequence,stop_id
0,1,0,0,8642
1,1,0,1,8652
2,1,0,2,8662
3,1,0,3,8672
4,1,0,4,8682


In [4]:
# for stop in dataframe.Stop.unique(): 
#     if len(dataframe[(dataframe.Stop==stop) & (dataframe.Real_date == 20210907)]) > 3: 
#         print(stop)

In [5]:
# dataframe[(dataframe.Stop=="1942") & (dataframe.Real_date == 20210907)]

In [6]:
def harmonic_mean(qualities):
    for key in qualities: 
        qualities[key] = len(qualities[key])/sum(1/quality for quality in qualities[key] )
    return qualities


def clear_zeros(dico: dict): 
    for key in dico.keys(): 
        dico[key] = [elem for elem in dico[key] if elem > 0]
    return dico

def sort_dict(dico) : 
    keys = list(dico.keys())
    values_ = list(dico.values())
    sorted_value_index = np.argsort(values_)
    sorted_dict = {keys[i]: values_[i] for i in sorted_value_index}
    return sorted_dict

def remove_empty(dico): 
    removed = []
    for key in dico.keys(): 
        if len(dico[key]) == 0: 
            removed.append(key)
    for elem in removed: 
        dico.pop(elem)
    return dico, removed

In [7]:
dataframe.head()

,Line,Stop,Date,Real_date,Score
0,12,1162C,20210901 - 20210917,20210906,90.18
1,12,1162C,20210901 - 20210917,20210907,89.29
2,12,1162C,20210901 - 20210917,20210908,87.52
3,12,1162C,20210901 - 20210917,20210909,90.64
4,12,1162C,20210901 - 20210917,20210910,94.08


In [8]:
stops = direction[(direction.direction_id == 0) & (direction.route_id==1) ].stop_id
dataframe[(dataframe.Real_date == 20210907) & (dataframe["Line"]==1) & (dataframe["Stop"].isin(stops))]

,Line,Stop,Date,Real_date,Score
23126,1,8012,20210901 - 20210917,20210907,90.16
23138,1,8022,20210901 - 20210917,20210907,90.94
23150,1,8032,20210901 - 20210917,20210907,92.96
23162,1,8042,20210901 - 20210917,20210907,91.70
23174,1,8052,20210901 - 20210917,20210907,92.62
23186,1,8062,20210901 - 20210917,20210907,91.71
23198,1,8072,20210901 - 20210917,20210907,91.47
23306,1,8272,20210901 - 20210917,20210907,89.47
23318,1,8282,20210901 - 20210917,20210907,88.02
23330,1,8292,20210901 - 20210917,20210907,93.12


In [9]:
weekday = [20210906, 20210907, 20210908, 20210909, 20210910,20210913, 20210914, 20210915, 20210916, 20210917, 20210911, 20210912]
lines = dataframe.Line.unique()
# stops = dataframe.Stop.unique()

scores = defaultdict(list)

def compute_average_score(dates=weekday, elems=lines, for_='Line'):
    for dir_ in [0,1]: 
        for line in elems: 
            stops = direction[(direction.direction_id==dir_) & (direction.route_id==line)].stop_id 
            for date in dates:
                scores[f"{line} - {date} - {dir_}"].extend(dataframe[(dataframe.Real_date == date) & (dataframe[f'{for_}']==line) & (dataframe.Stop.isin(stops))].Score)
    save = scores.copy()
    scores_w_zeros = clear_zeros(scores)
    scores_w_zeros, removed = remove_empty(scores_w_zeros)

    mean_scores = harmonic_mean(scores_w_zeros)

    sorted_mean_score = sort_dict(mean_scores)
    
    return sorted_mean_score, save

res, save_res = compute_average_score()

In [10]:
save_res.values()

dict_values([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [92.29], [89.77], [86.88], [90.92], [90.93], [90.57], [91.96], [88.06], [91.11], [89.39], [92.95], [86.73], [], [], [], [], [], [], [], [], [], [], [], [], [63.17, 67.44, 70.41, 69.41, 68.17, 65.42, 66.84, 60.99, 66.03, 53.89, 61.46, 61.26, 66.46, 60.5, 57.81, 60.64, 59.78, 59.74, 60.24, 60.41], [21.94, 93.03, 78.79, 91.77, 92.74, 91.9, 91.25, 76.35, 88.94, 91.06, 80.56, 78.49, 91.9, 78.02, 89.54, 77.0, 75.07, 77.6, 61.04, 31.9], [0.9, 88.4, 94.24, 92.15, 92.85, 87.54, 90.08, 79.68, 91.34, 86.74, 70.7, 76.74, 93.25, 76.18, 88.59, 59.89, 76.56, 73.92, 74.18, 53.94], [20.53, 88.19, 90.33, 94.29, 93.03, 90.31, 90.88, 81.0, 87.91, 90.23, 80.06, 78.93, 91.31, 78.71, 88.35, 79.8, 79.18, 71.72, 77.18, 44.1], [14.47, 91.44, 95.15, 94.41, 93.19, 91.11, 90.44, 80.94, 91.89, 90.39, 80.46, 79.34, 92.08, 77.44, 86.07, 77.29, 74.53, 78.96, 72.18

In [11]:
len(lines)

54

In [12]:
result_dataframe = pd.DataFrame(columns=["Line", "Date", "Direction", "Score"])

In [13]:
for elem in res.keys(): 
    values = []
    values = elem.split(' - ')
    values.append(res[elem])
    new_row = pd.Series(dict(zip(["Line", "Date", "Direction", "Score"], values)))
    result_dataframe = pd.concat([result_dataframe, new_row.to_frame().T], ignore_index=True)


    

In [28]:
result_dataframe.head()

,Line,Date,Direction,Score
0,48,20210909,1,0.857496
1,48,20210906,0,1.91943
2,48,20210906,1,4.39
3,48,20210912,1,5.215517
4,48,20210910,1,5.870743


In [30]:
result_dataframe.Score = result_dataframe.Score.apply(lambda row: round(row, 2))

In [31]:
result_dataframe.head()

,Line,Date,Direction,Score
0,48,20210909,1,0.86
1,48,20210906,0,1.92
2,48,20210906,1,4.39
3,48,20210912,1,5.22
4,48,20210910,1,5.87


In [32]:
result_dataframe.to_csv("scores_line.csv", index=False)

In [16]:
test = save_res['1 - 20210907 - 0']
qualities = len(test)/sum(1/quality for quality in test)

In [17]:
qualities

91.40727139778659

In [18]:
dataframe[dataframe.Stop == '2667F'].Line

22417    82
22418    82
22419    82
22420    82
22421    82
22422    82
22423    82
22424    82
22425    82
22426    82
22427    82
22428    82
22717    97
22718    97
22719    97
22720    97
22721    97
22722    97
22723    97
22724    97
22725    97
22726    97
22727    97
22728    97
Name: Line, dtype: int64

In [19]:
res

{'48 - 20210909 - 1': 0.8574956011544329,
 '48 - 20210906 - 0': 1.9194299287410925,
 '48 - 20210906 - 1': 4.39,
 '48 - 20210912 - 1': 5.215517482524033,
 '48 - 20210910 - 1': 5.870742922941095,
 '48 - 20210915 - 1': 6.191413678685264,
 '48 - 20210913 - 1': 6.3785420869091185,
 '48 - 20210911 - 1': 7.760545338009244,
 '48 - 20210914 - 1': 8.448649711432957,
 '48 - 20210908 - 1': 8.562678740949071,
 '48 - 20210907 - 1': 8.566060381935772,
 '48 - 20210916 - 1': 9.152788344158287,
 '48 - 20210917 - 1': 9.268315339240248,
 '48 - 20210912 - 0': 12.422219040801716,
 '34 - 20210914 - 0': 13.537194137180686,
 '27 - 20210908 - 0': 14.816692852848798,
 '27 - 20210908 - 1': 15.016511857720351,
 '29 - 20210909 - 0': 17.959291681108294,
 '29 - 20210909 - 1': 19.136465818678015,
 '48 - 20210910 - 0': 23.555917071537568,
 '61 - 20210912 - 0': 27.590614337652656,
 '61 - 20210912 - 1': 28.891485902093063,
 '43 - 20210913 - 1': 33.44078823053043,
 '27 - 20210912 - 1': 35.92625513911897,
 '48 - 20210913 -

In [20]:
res

{'48 - 20210909 - 1': 0.8574956011544329,
 '48 - 20210906 - 0': 1.9194299287410925,
 '48 - 20210906 - 1': 4.39,
 '48 - 20210912 - 1': 5.215517482524033,
 '48 - 20210910 - 1': 5.870742922941095,
 '48 - 20210915 - 1': 6.191413678685264,
 '48 - 20210913 - 1': 6.3785420869091185,
 '48 - 20210911 - 1': 7.760545338009244,
 '48 - 20210914 - 1': 8.448649711432957,
 '48 - 20210908 - 1': 8.562678740949071,
 '48 - 20210907 - 1': 8.566060381935772,
 '48 - 20210916 - 1': 9.152788344158287,
 '48 - 20210917 - 1': 9.268315339240248,
 '48 - 20210912 - 0': 12.422219040801716,
 '34 - 20210914 - 0': 13.537194137180686,
 '27 - 20210908 - 0': 14.816692852848798,
 '27 - 20210908 - 1': 15.016511857720351,
 '29 - 20210909 - 0': 17.959291681108294,
 '29 - 20210909 - 1': 19.136465818678015,
 '48 - 20210910 - 0': 23.555917071537568,
 '61 - 20210912 - 0': 27.590614337652656,
 '61 - 20210912 - 1': 28.891485902093063,
 '43 - 20210913 - 1': 33.44078823053043,
 '27 - 20210912 - 1': 35.92625513911897,
 '48 - 20210913 -

In [21]:
# line_day_scores = defaultdict(list)
# # values_without_zeros = defaultdict(list)
# for line in lines: 
#     for date in dates:
#         line_day_scores[f"{line} - {date}"].extend(dataframe[(dataframe.Real_date == date) & (dataframe.Line==line)].Score)

In [22]:
# line_day_scores_w_zeros = defaultdict(list)
# for key in line_day_scores.keys(): 
#     line_day_scores_w_zeros[key].extend(clear_zeros(line_day_scores[key]))

# line_day_scores_w_zeros, removed = remove_empty(line_day_scores_w_zeros)

In [23]:
# dico = {"a": [1,2,3,3], "b":[2,3,4,5]}
# dico = harmonic_mean(dico)
# dico

In [24]:
# line_day_score_average = {}
# for key in line_day_scores_w_zeros.keys():
#     print(key)
#     line_day_score_average[key] = harmonic_mean(line_day_scores_w_zeros[key])

In [25]:
sorted_line_day_average = sort_dict(line_day_score_average)

NameError: name 'line_day_score_average' is not defined

In [ ]:
sorted_line_day_average

{'97 - 20210917': 0.019808714768611124,
 '97 - 20210914': 0.02993805074561004,
 '97 - 20210908': 0.033430300019696234,
 '97 - 20210910': 0.037147689280314075,
 '97 - 20210913': 0.03831044845917253,
 '97 - 20210916': 0.04141031986383098,
 '97 - 20210909': 0.04357767809265846,
 '97 - 20210915': 0.04445573876107417,
 '97 - 20210907': 0.05052822889224738,
 '48 - 20210909': 0.08502087190181169,
 '81 - 20210914': 0.16785348930838434,
 '78 - 20210909': 0.26180596644219833,
 '79 - 20210913': 0.3393333338136616,
 '79 - 20210910': 0.37669316079789217,
 '8 - 20210908': 0.43498610535286863,
 '43 - 20210908': 0.45190504851661695,
 '79 - 20210917': 0.4647653322832447,
 '43 - 20210916': 0.4981158884032257,
 '29 - 20210909': 0.5055245161707755,
 '48 - 20210910': 0.5107056668577341,
 '43 - 20210910': 0.5573944808141001,
 '43 - 20210915': 0.5710322558529325,
 '79 - 20210914': 0.5796675728121085,
 '48 - 20210915': 0.5820037570135279,
 '34 - 20210914': 0.5831326899541776,
 '43 - 20210913': 0.5920474667200

In [ ]:
average_score = {}
average_score_without_zeros={}
for elem in values.keys(): 
    # print(f"The average score of the line {elem} is: {np.mean(values[elem])}")
    average_score[elem]= np.mean(values[elem])
    average_score_without_zeros[elem] = harmonic_mean(values_without_zeros[elem])


In [ ]:


sort_average_score = sort_dict(average_score)
sort_average_score_without_zeros = sort_dict(average_score_without_zeros)



In [ ]:
sort_average_score

{'97': 0.5017156862745099,
 '21': 13.591818181818182,
 '48': 27.072777777777773,
 '51': 30.65547619047619,
 '19': 39.71851063829787,
 '46': 51.82402777777778,
 '80': 54.27682291666667,
 '77': 55.49677777777778,
 '69': 62.55094736842104,
 '70': 65.33364055299539,
 '72': 66.9925080385852,
 '76': 67.85719387755101,
 '57': 72.0250895140665,
 '33': 73.00430555555556,
 '43': 73.66238591916559,
 '74': 75.5309677419355,
 '27': 76.92480167014612,
 '56': 77.20022321428571,
 '41': 77.21426153846154,
 '12': 77.51947916666667,
 '61': 78.43049222797927,
 '37': 78.65072463768115,
 '14': 78.77940476190476,
 '13': 79.92609699769054,
 '47': 80.6296682464455,
 '79': 80.63381249999999,
 '36': 81.46691218130312,
 '89': 81.54221115537848,
 '78': 81.68541666666667,
 '83': 82.59735294117647,
 '20': 82.7218472906404,
 '42': 82.87360000000001,
 '59': 83.30085106382978,
 '34': 83.77963414634146,
 '88': 83.89378787878788,
 '95': 84.17161739130435,
 '86': 85.61871964679914,
 '7': 85.76777777777778,
 '8': 85.851575

In [ ]:
values_without_zeros['21']

[49.85, 70.3, 29.36]

In [ ]:
sort_average_score_without_zeros

{'97': 0.0038235819293471396,
 '48': 0.03848064783890331,
 '79': 0.04863518769065308,
 '43': 0.05013481884370016,
 '81': 0.06711477766392863,
 '29': 0.09137811632827068,
 '8': 0.09348934421248692,
 '82': 0.09536770604003027,
 '7': 0.09780599811648331,
 '74': 0.1003256544111364,
 '83': 0.1100785776991978,
 '41': 0.11052239875759434,
 '56': 0.11066923584294279,
 '36': 0.11281968732273595,
 '87': 0.11472772748869356,
 '78': 0.11528233220172779,
 '88': 0.12035324441181736,
 '27': 0.1215969656450346,
 '14': 0.12422925057169072,
 '34': 0.12542893338973649,
 '59': 0.12544122959228432,
 '70': 0.13126808251042948,
 '5': 0.13389956859187285,
 '95': 0.13641168217924787,
 '37': 0.14074582424400248,
 '6': 0.1446109468956049,
 '13': 0.14485645283767315,
 '64': 0.14758545971909814,
 '38': 0.15060873794019744,
 '86': 0.15459674281414296,
 '89': 0.16151362300601677,
 '47': 0.16431764146429434,
 '39': 0.17158914647827944,
 '1': 0.17281923167329602,
 '57': 0.1773027919156365,
 '63': 0.1796774537551502,
 